# Elastic Stackの構築手順

## 1サーバでの構築

本ドキュメントは、1サーバ上にLogstashとElasticsearchを構築する手順となります。

### 構成と手順の概要

1サーバ上にElasticsearchを構築します。  

このNotebookで構成するのは、以下の図に示すような構成です。 
各ノードの役割は[01_Setup](01_Setup.ipynb#ノードの役割の違い)を参照してください。

![images/01_1server.png](images/01_1server.png)

この後のNotebookでの大まかな手順は次の通りです。  

1. jdkのインストール
2. Elasticsearchのインストール
3. elasticsearch.ymlの設定
4. Elasticsearchの起動
5. Logstashのインストール
6. Logstashの設定
7. サンプルデータの投入

なおLogstashとは、Elastic Stackを構成するプロダクトの１つです。  
様々な形式のデータの読み込み、加工、格納を処理するパイプラインを簡単に構築できます。  
  
詳細は[04_Store_Data](04_Store_Data.ipynb)で説明しますが、ここではサンプルデータを投入したいので、先にインストールまでしておきます。

### 準備

ansibleから構築対象サーバへ接続確認を行います。  
このNotebookで環境を構築したい先のサーバを、次のセルで設定してください。

In [118]:
%env ES_CLIENT=XXX.XXX.XXX.231:9200
%env LOGSTASH_HOST=XXX.XXX.XXX.231
%env INDEX=meteorological-data-*
%env TYPE=logs

env: ES_CLIENT=XXX.XXX.XXX.231:9200
env: LOGSTASH_HOST=XXX.XXX.XXX.231
env: INDEX=meteorological-data-*
env: TYPE=logs


各サーバにansibleコマンドを発行するため、
ansibleを実行可能なユーザアカウントとそのSSHキーを次のセルで設定してください。

In [103]:
#elasticsearchをインストールするサーバーでansibleコマンドを実行するユーザー
USER='ansible'

#公開鍵認証を行う場合の秘密鍵のパス
KEYPATH='~/.ssh/ansible_id_rsa'

ansibleのインベントリファイルを作成します。  
次のセルの内容を/etc/ansible/hostsとして保存します。

[es-node]  
< ES_CLIENT >  
[logstash-server]  
< LOGSTASH_HOST >

ここまでの設定が問題ないか確認します。  
次のセルを実行して、ansibleコマンドが各サーバで実行できることを確認してください。

In [104]:
!ansible all -m ping -i /etc/ansible/host-1server -u $USER --private-key=$KEYPATH

XXX.XXX.XXX.231 | SUCCESS => {
    "changed": false, 
    "ping": "pong"
}


各サーバから、次のようにSUCCESSという応答が返ってくれば問題なく実行できています。

この後から実際の構築作業を開始します。

### jdkのインストール

ElasticsearchはJavaで実装されており、実行するためにはJavaの環境が必要です。  
Elasticsearch-5.0.0ではバージョン1.8.0_73以降のものが推奨されています。

次のコマンドを実行して、jdkをインストールします。

In [105]:
!ansible-playbook -i /etc/ansible/host-1server playbooks/install_jdk_1server.yml -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.231]

TASK [download jdk-8u112-linux-x64.rpm] ****************************************
changed: [XXX.XXX.XXX.231]

TASK [install jdk] *************************************************************
changed: [XXX.XXX.XXX.231]

TASK [set JAVA_HOME] ***********************************************************
ok: [XXX.XXX.XXX.231]

TASK [set PATH] ****************************************************************
ok: [XXX.XXX.XXX.231]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.231              : ok=5    changed=2    unreachable=0    failed=0   



### Elasticsearchのインストール

次のコマンドを実行して、Elasticsearchをインストールします。

In [106]:
!ansible-playbook playbooks/install_elasticsearch_1server.yml -i /etc/ansible/host-1server -u $USER --private-key=$KEYPATH


PLAY [all] *********************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.231]

TASK [install elasticsearch 5.0.0] *********************************************
changed: [XXX.XXX.XXX.231]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.231              : ok=2    changed=1    unreachable=0    failed=0   



### elasticsearch.ymlの設定

1ノードで動作するための設定を行います。

elasticsearch.ymlの設定値を次の表に記載します。  
デフォルトファイルの変更箇所を示すので、環境に合わせて適宜変更してください。  
パスは/etc/elasticsearch/elasticsearch.ymlです。

| パラメータ名                       | Node           |
|----------------------------------|----------------|
| node.name                        | ホスト名        |
| network.host                     | IPアドレス      |

#### 設定項目の説明

* node.name: ホスト名  
ノードの名前です。ホスト名を指定してください。  

  
* network.host: IPアドレス  
他のノードからアクセスする際に指定するホスト名またはIPアドレスを示します。  
ノード自身のホスト名またはIPアドレスで書きかえてください。  

### Elasticsearchの起動

Elasticsearchを起動します。

In [120]:
!ansible-playbook playbooks/start_1server.yml -i /etc/ansible/host-1server -u $USER --private-key=$KEYPATH


PLAY [es-node] *****************************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.231]

TASK [start elasticsearch 1server] *********************************************
changed: [XXX.XXX.XXX.231]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.231              : ok=2    changed=1    unreachable=0    failed=0   



各サーバの起動は次のコマンドで確認してください。

In [121]:
!ansible all -i /etc/ansible/host-1server -m shell -a "sudo systemctl status elasticsearch" -u $USER --private-key=$KEYPATH

XXX.XXX.XXX.231 | SUCCESS | rc=0 >>
● elasticsearch.service - Elasticsearch
   Loaded: loaded (/usr/lib/systemd/system/elasticsearch.service; disabled; vendor preset: disabled)
   Active: active (running) since Wed 2016-12-14 11:26:50 JST; 1h 5min ago
     Docs: http://www.elastic.co
 Main PID: 19632 (java)
   CGroup: /system.slice/elasticsearch.service
           └─19632 /bin/java -Xms2g -Xmx2g -XX:+UseConcMarkSweepGC -XX:CMSInitiatingOccupancyFraction=75 -XX:+UseCMSInitiatingOccupancyOnly -XX:+DisableExplicitGC -XX:+AlwaysPreTouch -server -Djava.awt.headless=true -Dfile.encoding=UTF-8 -Djna.nosys=true -Dio.netty.noUnsafe=true -Dio.netty.noKeySetOptimization=true -Dlog4j.shutdownHookEnabled=false -Dlog4j2.disable.jmx=true -Dlog4j.skipJansi=true -XX:+HeapDumpOnOutOfMemoryError -Des.path.home=/usr/share/elasticsearch -cp /usr/share/elasticsearch/lib/elasticsearch-5.0.0.jar:/usr/share/elasticsearch/lib/* org.elasticsearch.bootstrap.Elasticsearch -p /var/run/elasticsearch/elasticsearch.pi

次のように  
**Active: active (running)**    
と表示されれば正常に起動しています。

### Logstashのインストール

次のコマンドを実行して、Logstashをインストールします。

In [109]:
!ansible-playbook -i /etc/ansible/host-1server playbooks/install_logstash_1server.yml -u $USER --private-key=$KEYPATH


PLAY [logstash-server] *********************************************************

TASK [setup] *******************************************************************
ok: [XXX.XXX.XXX.231]

TASK [install logstash] ********************************************************
changed: [XXX.XXX.XXX.231]

PLAY RECAP *********************************************************************
XXX.XXX.XXX.231              : ok=2    changed=1    unreachable=0    failed=0   



### Logstashの設定

サンプルデータをElasticsearchに登録するためのlogstash設定ファイルは[logstash_conf](logstash_conf)に配置してあるmain.confを用います。  
main.confの中で、Elasticsearchのホスト名を指定する箇所があるので、  
[準備](#準備)で定義した  

ES_CLIENT:< ES_CLIENT >  

で書き換えてください。  
具体的な修正箇所は次の通りです


ここで利用するmain.confは標準入力で読み込んだCSV形式の内容をElasticsearchに格納するための設定です。  
具体的な意味は[04_Store_Data](04_Store_Data.ipynb)で説明します。

main.confをLogstashのサーバーの/etc/logstash/conf.d配下にコピーしてください。

### データ型の定義（マッピング定義）

サンプルデータの登録をする前に、登録するデータの「データ型」を定義しておく必要があります。  
データ型の種類およびマッピング定義の詳細に関しては[05_Indexing](http://localhost:8888/notebooks/05_Indexing.ipynb)を参照してください。

次のコマンドを実行してマッピング定義を行います。

In [123]:
%%bash
curl -XPUT "http://$ES_CLIENT/_template/weather" -d @- << EOF
{
  "template" : "meteorological-data-*",
  "mappings": {
    "logs": {
      "dynamic_templates" : [
        {
          "my_strings" : {
            "match_mapping_type" : "string",
            "mapping" : {
              "type" : "keyword"
            }
          }
        }
      ],
      "properties": {
        "atmospheric_pressure": {
          "type": "float"
        },
        "sea_level_pressure":{
          "type": "float"
        },
        "precipitation_day": {
          "type": "integer"
        },
        "precipitation_day": {
          "type": "float"
        },
        "precipitation_max_hour":{
          "type":"float"
        },
        "precipitation_max_10min":{
          "type":"float"
        },
        "temperature_avg":{
          "type":"float"
        },
        "temperature_max":{
          "type":"float"
        },
        "temperature_min":{
          "type":"float"
        },
        "humidity_avg":{
          "type":"float"
        },
        "humidity_min":{
          "type":"float"
        },
        "wind_speed_avg":{
          "type":"float"
        },
        "wind_speed_max":{
          "type":"float"
        },
        "wind_speed_max_moment":{
          "type":"float"
        },
        "sunshine_duration":{
          "type":"float"
        },
        "snowfall":{
          "type":"float"
        },
        "snowfall_max":{
          "type":"float"
        }
      }
    }
  }
}
EOF

{"acknowledged":true}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  1394  100    21  100  1373    152   9970 --:--:-- --:--:-- --:--:-- 10021


### サンプルデータの投入

以降の章で扱うサンプルデータを登録します。  
ここでは気象庁から提供されている、東京都の2015年における気象データ1年分を用います。

[気象庁「過去の気象データ」](http://www.data.jma.go.jp/gmd/risk/obsdl/)

登録するデータの内容は次のコマンドで確認できます。

In [124]:
!head -n 5 sample_data/tokyo2015.csv

Logstashのサーバーにサンプルデータをコピーします。  

[sample_data](sample_data)にあるtokyo2015.csvをLogstashのサーバーに配置してください。

Logstashを使ってサンプルデータをElasticsearchに登録します。次のセルのコマンドを実行してください。

In [125]:
!ansible logstash-server -i /etc/ansible/host-1server -m shell -a "cat tokyo2015.csv | sudo /usr/share/logstash/bin/logstash --path.settings /etc/logstash -f /etc/logstash/conf.d/main.conf" -u $USER --private-key=$KEYPATH

XXX.XXX.XXX.231 | SUCCESS | rc=0 >>
Sending Logstash logs to /var/log/logstash which is now configured via log4j2.properties.



データが登録されているかどうかは、次のセルを実行することで確認できます。

In [126]:
%%bash
curl -XGET "http://$ES_CLIENT/$INDEX/_search/?pretty"

{
  "took" : 49,
  "timed_out" : false,
  "_shards" : {
    "total" : 70,
    "successful" : 70,
    "failed" : 0
  },
  "hits" : {
    "total" : 358,
    "max_score" : 1.0,
    "hits" : [
      {
        "_index" : "meteorological-data-2015.01",
        "_type" : "logs",
        "_id" : "tokyo_15-01-08",
        "_score" : 1.0,
        "_source" : {
          "date" : "15-01-08",
          "wind_speed_avg" : "4.4",
          "snowfall" : null,
          "temperature_max" : "11.2",
          "precipitation_day" : null,
          "wind_speed_max" : "8.4",
          "sunshine_duration" : "8.9",
          "temperature_avg" : "6.2",
          "temperature_min" : "2.0",
          "@version" : "1",
          "wind_direction_max_moment" : "北西",
          "information_night" : "晴",
          "precipitation_max_10min" : null,
          "precipitation_max_hour" : null,
          "wind_direction" : "西",
          "information_daytime" : "晴",
          "@timestamp" : "2015-01-07T15:00:00.000Z",
  

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 11287  100 11287    0     0   132k      0 --:--:-- --:--:-- --:--:--  134k
